# Chapter 08 실습: 모델 저장과 배포 파이프라인

## 실습 목표
- MNIST CNN 모델을 학습하고 `.keras` 형식으로 저장한다
- 저장된 모델을 로드하여 테스트 정확도를 검증한다
- TFLite로 변환 후 인터프리터 추론을 실행한다
- 원본 모델 vs TFLite의 크기/속도/정확도를 비교한다

## 도전 과제
마지막 셀의 표를 직접 작성해 보세요!

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import os
import time

print(f"TensorFlow 버전: {tf.__version__}")

# 결과물 저장 디렉토리
OUTPUT_DIR = '/tmp/ch08_practice'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"저장 경로: {OUTPUT_DIR}")

In [ ]:
# MNIST CNN 모델 학습

# 데이터 로드 및 전처리
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0
X_train = X_train[..., np.newaxis]  # 채널 차원 추가
X_test  = X_test[..., np.newaxis]

print(f"학습 데이터: {X_train.shape} | 테스트 데이터: {X_test.shape}")

# 간단한 CNN 모델 구성
model = tf.keras.Sequential([
    # 첫 번째 합성곱 블록
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1),
                           name='conv1'),
    tf.keras.layers.MaxPooling2D(2, 2, name='pool1'),
    # 두 번째 합성곱 블록
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    tf.keras.layers.MaxPooling2D(2, 2, name='pool2'),
    # 분류기 헤드
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(128, activation='relu', name='dense1'),
    tf.keras.layers.Dropout(0.3, name='dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
], name='mnist_cnn')

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# 학습 실행
print("\n모델 학습 시작...")
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=128,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
    ]
)
print("학습 완료")

In [ ]:
# .keras 저장 → 로드 → 테스트 정확도 확인

# 원본 모델 테스트 정확도 측정
start = time.perf_counter()
orig_loss, orig_acc = model.evaluate(X_test, y_test, verbose=0)
orig_eval_time = (time.perf_counter() - start) * 1000
print(f"[원본] 테스트 정확도: {orig_acc:.4f} | 평가 시간: {orig_eval_time:.1f} ms")

# .keras 형식으로 저장
keras_save_path = os.path.join(OUTPUT_DIR, 'mnist_cnn.keras')
model.save(keras_save_path)
keras_size = os.path.getsize(keras_save_path)
print(f"\n.keras 저장 완료: {keras_save_path}")
print(f".keras 파일 크기: {keras_size:,} bytes ({keras_size/1024:.1f} KB)")

# 모델 로드 후 재평가
loaded_model = tf.keras.models.load_model(keras_save_path)
loaded_loss, loaded_acc = loaded_model.evaluate(X_test, y_test, verbose=0)
print(f"\n[로드] 테스트 정확도: {loaded_acc:.4f}")
print(f"원본 vs 로드 정확도 일치: {abs(orig_acc - loaded_acc) < 1e-6}")

# 예측값 일치 확인
orig_preds   = model.predict(X_test[:10], verbose=0)
loaded_preds = loaded_model.predict(X_test[:10], verbose=0)
print(f"예측값 수치 일치: {np.allclose(orig_preds, loaded_preds)}")

In [ ]:
# TFLite 변환 → 인터프리터 추론 → 정확도 비교

# ── TFLite FP32 변환 ──────────────────────────────────────────────
converter_fp32 = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_fp32 = converter_fp32.convert()
path_fp32 = os.path.join(OUTPUT_DIR, 'mnist_fp32.tflite')
with open(path_fp32, 'wb') as f:
    f.write(tflite_fp32)

# ── TFLite Dynamic Quantization 변환 ─────────────────────────────
converter_dq = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter_dq.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dq = converter_dq.convert()
path_dq = os.path.join(OUTPUT_DIR, 'mnist_dynamic.tflite')
with open(path_dq, 'wb') as f:
    f.write(tflite_dq)

print("TFLite 변환 완료")
print(f"  FP32    : {os.path.getsize(path_fp32)/1024:.1f} KB")
print(f"  Dynamic : {os.path.getsize(path_dq)/1024:.1f} KB")

# ── TFLite 인터프리터 추론 함수 ───────────────────────────────────
def evaluate_tflite(tflite_path, images, labels, n=1000):
    """
    TFLite 인터프리터로 추론을 실행하고 정확도와 총 시간을 반환한다.
    단일 이미지 추론으로 실제 엣지 디바이스 환경을 시뮬레이션한다.
    """
    interp = tf.lite.Interpreter(model_path=tflite_path)
    interp.allocate_tensors()  # 텐서 메모리 할당
    input_idx  = interp.get_input_details()[0]['index']
    output_idx = interp.get_output_details()[0]['index']

    correct = 0
    start = time.perf_counter()
    for i in range(n):
        # 단일 이미지 배치로 변환 [1, 28, 28, 1]
        inp = images[i:i+1]
        interp.set_tensor(input_idx, inp)
        interp.invoke()  # 추론 실행
        out = interp.get_tensor(output_idx)
        if np.argmax(out[0]) == labels[i]:
            correct += 1
    total_ms = (time.perf_counter() - start) * 1000

    return correct / n, total_ms, total_ms / n

# 추론 실행 (각 1000개 샘플)
print("\nTFLite 추론 실행 중...")
acc_fp32, total_fp32, avg_fp32 = evaluate_tflite(path_fp32, X_test, y_test)
acc_dq,   total_dq,   avg_dq   = evaluate_tflite(path_dq,   X_test, y_test)

print(f"FP32    - 정확도: {acc_fp32:.4f} | 평균 추론: {avg_fp32:.3f} ms")
print(f"Dynamic - 정확도: {acc_dq:.4f}   | 평균 추론: {avg_dq:.3f} ms")

# 결과 딕셔너리 (마지막 셀의 표 작성에 활용)
results = {
    'original_keras': {
        'size_kb': keras_size / 1024,
        'accuracy': orig_acc,
        'eval_time_ms': orig_eval_time
    },
    'tflite_fp32': {
        'size_kb': os.path.getsize(path_fp32) / 1024,
        'accuracy': acc_fp32,
        'avg_infer_ms': avg_fp32
    },
    'tflite_dynamic': {
        'size_kb': os.path.getsize(path_dq) / 1024,
        'accuracy': acc_dq,
        'avg_infer_ms': avg_dq
    }
}
print("\n결과 딕셔너리 저장 완료 (results 변수)")

## 도전 과제: 비교 표 작성

위 `results` 딕셔너리의 값을 이용하여 아래 표를 완성하세요.

### 결과 비교 표 (직접 채워보기)

| 형식 | 크기 (KB) | 테스트 정확도 | 단일 추론 시간 (ms) | 비고 |
|------|-----------|--------------|--------------------|---------|
| 원본 `.keras` | ? | ? | ? (전체 배치) | 학습 가능 |
| TFLite FP32 | ? | ? | ? | 변환만 수행 |
| TFLite Dynamic Quant | ? | ? | ? | 가중치 int8 |

### 분석 질문

1. Dynamic Quantization 적용 후 모델 크기는 몇 % 감소했나요?
2. 정확도 손실은 얼마나 발생했나요? 허용 가능한 수준인가요?
3. TFLite FP32와 Dynamic Quant의 추론 속도 차이는 어떤가요?
4. 실제 모바일 배포 시 어떤 형식을 선택하겠습니까? 이유는?

### 추가 도전

- Float16 Quantization도 추가하여 비교해보세요
- `results` 딕셔너리를 사용해 Python 코드로 표를 자동 출력하는 셀을 추가해보세요